Below block defines the function for getting a number of search results from immoweb.

In [44]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import random
from random import randint
import re

def get_search_results(maxpage):
    """Collect search results running up to the maxpage number of both newest 'house'  and 'appartment' sales,
    returning a dictionary of {"hyperlink":[],"house_is":[]}"""
    # initialise the dictionary with the results
    search_results = {"hyperlink":[],"house_is":[]}
    # start the loop
    for i in range(1, maxpage+1):
        # for each loop, scrape one results page of houses and one of appartments
        # the results are added if they are not there yet
        for houselink in results_page_scrape(i,"house"):
            if houselink not in search_results["hyperlink"]:
                search_results["hyperlink"].append(houselink)
                search_results["house_is"].append(1)
        for apartmentlink in results_page_scrape(i,"apartment"):
            if apartmentlink not in search_results["hyperlink"]:
                search_results["hyperlink"].append(apartmentlink)
                search_results["house_is"].append(0)
    return search_results

def results_page_scrape(pagenumber,propertytype):
    '''A subroutine scraping links from search results on a propertytype/page'''
    # initialise the return
    links = []
    # I slow down the frequency of requests to avoid being identified and therefore ban from the site
    time.sleep(random.uniform(1.0, 2.0))
    # setup the selenium webdriver; if able to find elements within the given
    # span it returns as soon as finding them, else it raises an exception after 10 seconds.
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    url=f'https://www.immoweb.be/en/search/{propertytype}/for-sale?countries=BE&isALifeAnnuitySale=false&page={pagenumber}&orderBy=newest'
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    for elem in soup.find_all('a', attrs={"class":"card__title-link"}):
        # get hyperlink to property page
        hyperlink = elem.get('href')
        # cut the searchID off
        hyperlink = re.match("(.+)\?searchId=.+", hyperlink).group(1)
        # append to the return
        links.append(hyperlink)
    driver.close()
    return links

Getting 12K resuls would need to run this function up to max page number 200. That would take about 1 hour.
Lets just do 2:

In [46]:
get_search_results(2)

{'hyperlink': ['https://www.immoweb.be/en/classified/house/for-sale/seraing/4100/8771411',
  'https://www.immoweb.be/en/classified/house/for-sale/ath/7800/8952732',
  'https://www.immoweb.be/en/classified/house/for-sale/seraing/4100/8952728',
  'https://www.immoweb.be/en/classified/house/for-sale/boechout/2530/8947153',
  'https://www.immoweb.be/en/classified/house/for-sale/marke/8510/8952714',
  'https://www.immoweb.be/en/classified/house/for-sale/mouscron/7700/8455936',
  'https://www.immoweb.be/en/classified/house/for-sale/heule/8501/8952709',
  'https://www.immoweb.be/en/classified/house/for-sale/aiseau-presles/6250/8946872',
  'https://www.immoweb.be/en/classified/house/for-sale/arlon/6700/8952699',
  'https://www.immoweb.be/en/classified/house/for-sale/scherpenheuvel/3270/8952698',
  'https://www.immoweb.be/en/classified/mixed-use-building/for-sale/bertrix/6880/8952696',
  'https://www.immoweb.be/en/classified/house/for-sale/ronse/9600/8952695',
  'https://www.immoweb.be/en/class